# Full OCRopus 4 Training and Running

This worksheet demonstrates OCRopus 4 training from start to finish, including bootstrapping using another OCR system.



Important Notes:

- this does NOT yield usable models since the training data set used in this worksheet is so small and limited
- this implements the main components of an OCR system and is suitable for text indexing and other applications, but full page recognition requires some additional components

In [ ]:
import os
os.getcwd()
os.system("sos convert full.ipynb full.sos")

In [ ]:
rm -f *.sqlite3 *.pth
rm -f book-seg.tar book-words.tar book-lines.tar

In [ ]:
[global]
display = 10
maxpages = 50; multiplier = 1
# maxpages = 9999; multiplier = 1000
maxextract = 1000 * multiplier
nwordtrain = 1000 * multiplier
nsegtrain = 200 * multiplier
nrottrain = 200 * multiplier
nskewtrain = 200 * multiplier
nscaletrain = 200 * multiplier
nbintrain = 10 * multiplier
dataurl = 'http://storage.googleapis.com/ocropus4/Volume_0000.tar'

# Download Dataset and Run Tesseract

The dataset is Volume 0 from Google's 1000 Book dataset. These images are camera scanned, modest resolution, modest quality images. This volume contains a mix of English and Gaelic.

_This dataset is used here for demonstration purposes only. It is not large or diverse enough to train a practical recognizer. You can obtain a usable recognizer by training on the UW3 dataset, but unfortunately that's not redistributable. We are currently putting together a larger, representative training set._

We use Tesseract as our bootstrap OCR system.

In [ ]:
[download]
input: []
output: 'Volume_0000.tar'
sh: expand=True
    curl {dataurl} > Volume_0000.tar

In [ ]:
%sosrun download

## Binarization

By default, we binarize with a hardcoded binarizer based on grayscale morphology. This binarizer is purely CPU based (we may eventually port it to the GPU). It has worked very well in many different settings. 

In [ ]:
tarp cat --help

In [ ]:
[binarize]
input: 'Volume_0000.tar'
output: 'raw.tar'
sh: expand=True
    vocropus nlbin binarize {_input} --gray --output raw.tar --maxrec {maxpages}

In [ ]:
%sosrun binarize

## Bootstrap OCR Recognizer

We're using Tesseract as the bootstrap OCR engine. Tesseract runs on the normalized images returned in the previous step. The bootstrap OCR recognizer can have a moderately high error rate and OCRopus 4 can still be trained on it successfully. In fact, once you have trained an initial OCRopus 4 recognizer, you can use it as the bootstrap recognizer for additional source materials. OCRopus 4 is, in effect, self-training and self-improving.

In [ ]:
!rebuild=y ./dtess --help

In [ ]:
[tess]
input: 'raw.tar'
output: 'tess.tar'
sh: expand=True
    rebuild=y dtess --help
    tarp proc -c $(pwd)/'dtess sample.nrm.jpg sample.tess -l eng hocr; rm -f sample.lines.png sample.pseg.png' {_input} -o {_output}.tar.temp
    mv {_output}.tar.temp {_output}

In [ ]:
%sosrun tess

# Generating Training Datasets

OCRopus 4 training starts with tar files containing pairs of images and corresponding hOCR output. These are used to generate training data for word and line recognition and page segmentation.

In [ ]:
%pylab inline

In [ ]:
figsize(12, 8)
import webdataset as wds
from itertools import islice

In [ ]:
ds = wds.WebDataset("tess.tar").decode("rgb8")
sample = next(islice(ds, 33, 999))
print(sample.keys())
print(sample["tess.hocr"].decode("utf-8"))

In [ ]:
figsize(15, 10)
imshow(sample["nrm.jpg"])

## Segmentation Data

Segmentation algorithms are usually trained on four-label segmentation targets consisting of markers, areas, separators, and background. The `extract-seg hocr2seg` subcommand extracts those training images.

Ordinarily, we would extract this data from many shards and shuffle it for training, but we're dispensing with that in this simple example.


In [ ]:
[seg-data]
input: 'tess.tar'
output: 'book-seg.tar'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus extract-seg hocr2seg {_input} --output {_output}.temp --element ocrx_word --extensions 'nrm.jpg tess.hocr' --invert True \
        --show 100 --labels='1, 2, 3' --check word --maxcount {maxextract}
    mv {_output}.temp {_output}


In [ ]:
%sosrun seg-data

In [ ]:
ds = wds.WebDataset("book-seg.tar").decode("rgb")
sample= next(islice(iter(ds), 5, 999))
print(sample.keys())
subplot(121); imshow(sample["png"])
subplot(122); imshow(sample["seg.png"][...,2])

## Word Training Data

As before, we use `extract-rec hocr2rec` to extract word training images.

In [ ]:
[word-data]
input: 'tess.tar'
output: 'book-words.tar'
sh: expand=True
    vocropus extract-rec hocr2rec {_input} --output {_output}.temp --element ocrx_word --extensions 'nrm.jpg tess.hocr' \
        --bounds 10,10,500,100 --dictionary /usr/share/dict/american-english-huge --maxcount {maxextract} \
        --acceptable-conf 60.0
    mv {_output}.temp {_output}

In [ ]:
%sosrun word-data

In [ ]:
figsize(12, 8)
import webdataset as wds
from itertools import islice
ds = wds.WebDataset("book-words.tar").decode("rgb")
start = 55
for i, sample in enumerate(islice(iter(ds), start, start+250, 10)):
    subplot(5, 5, i+1)
    imshow(sample["png"])
    title(sample["txt"])

## Detecting Character Outliers

We need to decide on a character training set. Here is some optional code that goes through all training strings and identifies rare and unusual characters.

You can use the output from this optional step either to transform the text strings in the training data, or to add new character set classes to OCRopus 4.

In [ ]:
[detect-outliers]
input: 'book-words.tar'
output: 'bad-chars.txt'

python:

    ds = wds.WebDataset("book-words.tar").decode("rgb")
    alltext = [sample["txt"] for sample in ds]

    from collections import Counter
    counter = Counter()
    for s in alltext: counter.update(s)

    for c, n in counter.most_common():
        if len(str(c.encode("unicode_escape"))) > 4:
            print(f"""{c} {str(c.encode("unicode_escape")):10s} {n:10d}""")

# Training a Word Segmenter

This is the training script for word segmentation. It's pretty straightforward and just uses the `book-seg.tar` data generated previously.

All training jobs log to Sqlite3 databases and also save their models into those databases. You can use the `ocropus4 slog` subcommand to extract the best or the last model and plot training curves.

In [ ]:
[train-segs]
input: 'book-seg.tar'
output: 'segtrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus ocroseg train {_input} --log-to segtrain.sqlite3 --ntrain {nsegtrain} --display {display}
    vocropus slog getlast segtrain.sqlite3 --output {_output}

In [ ]:
%sosrun train-segs

# Training a Word Recognizer

To train a word recognizer, we use the `ocrorec train` command using the `book-words.tar` dataset.

In [ ]:
[train-words]
input: 'book-words.tar'
output: 'wordtrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus ocrorec train {_input} --log-to wordtrain.sqlite3 --ntrain {nwordtrain} --display {display}
    vocropus slog getlast wordtrain.sqlite3 --output {_output}

In [ ]:
%sosrun train-words

# Line Segmentation

The base recognizer recognizes individual words; these need to be grouped into text lines as part of the recognition process. We train a separate text line segmentation model for this. The procedure is the same as for word segmentation: we generate line segmentation images for training and then use the `ocroseg train` command to train a segmenter.

In [ ]:
[line-data]
input: 'tess.tar'
output: 'book-lines.tar'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus extract-seg hocr2seg {_input} --output {_output}.temp --element ocr_line --extensions 'nrm.jpg tess.hocr' --invert True \
        --show 100 --labels='1, 2, 3' --check line --maxcount {maxextract}
    mv {_output}.temp {_output}


In [ ]:
%sosrun line-data

In [ ]:
ds = wds.WebDataset("book-lines.tar").decode("rgb")
sample= next(islice(iter(ds), 100, 999))
print(sample.keys())
subplot(121); imshow(sample["png"])
subplot(122); imshow(sample["seg.png"][...,2])

In [ ]:
[train-lines]
input: 'book-lines.tar'
output: 'linetrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus ocroseg train {_input} --log-to linetrain.sqlite3 --ntrain {nsegtrain} --display {display}
    rm -f {_output}
    vocropus slog getlast linetrain.sqlite3 --output {_output}

In [ ]:
%sosrun train-lines

# Running the Page Recognizer

The page reocgnizer consists of several steps:

- word segmentation: segment the input page into individual words
- word recognition: apply the word recognizer to the word segments
- line segmentation: apply the line segmenter to obtain text line seeds
- grouping: use the line segmentation output to group word boxes into text line
- sorting: text lines are finally sorted in a simple way to obtain the final output

Output is both in .tar format and as a single .html file in hOCR format (the hOCR is not compliant yet).

The final sort currently uses simple heuristics; eventually, there will be a second grouping step based on text block training.

In [ ]:
[page-recognition]
input: 'tess.tar'
output: 'recognized.tar'
sh: expand=True
    export MPLBACKEND=TkAgg
    vocropus pagerec recognize {_input} --output {_output} --recmodel wordtrain.pth --segmodel segtrain.pth --lgmodel linetrain.pth --full-html recognized.html --extensions nrm.jpg --maxrec 10

In [ ]:
%sosrun page-recognition

Here are some recognition results; these are NOT representative of what is achievable with OCRopus 4, since they are recognition results obtained on a tiny training data set with limited training data. Given this very limited training, recognition performance is actually pretty good.

In [ ]:
!lynx -dump recognized.html | tail -30

# Page Orientation

Before any kind of document processing, it is a good idea to ensure that pages are right side up. The `ocrorot` command handles this. We can first train a page orientation model and then apply it to correct the orientation of pages. Training just requires a collection of pages in right-side-up orientation as input.

In [ ]:
[train-rot]
input: 'tess.tar'
output: 'rottrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    base=$(basename {_output} .pth)
    rm -f $base.sqlite3
    vocropus ocrorot train {_input} --display 15 --output $base.sqlite3 --nsamples {nrottrain} --replicate 8 --num-workers 8 --extensions nrm.jpg
    vocropus slog getbest $base.sqlite3 --output {_output}

In [ ]:
%sosrun train-rot

In [ ]:
[predict-rot]
input: 'rottrain.pth'
sh: expand=True
    vocropus ocrorot correct tess.tar --nsamples 10 --model rottrain.pth

In [ ]:
%sosrun predict-rot

# Page Skew Correction

"Page skew" refers to small rotations of scanned pages--small deviations from the vertical. The `ocroskew` command can be used to train page skew models and perform page skew correction. These models are generally trained and applied after page orientation correction. Training just requires a collection of already deskewed images as input.

In [ ]:
[train-skew]
input: 'tess.tar'
output: 'skewtrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    base=$(basename {_output} .pth)
    rm -f $base.sqlite3
    vocropus ocroskew train {_input} --display 15 --output $base.sqlite3 --nsamples {nskewtrain} --replicate 8 --num-workers 8 --extensions 'nrm.jpg'
    vocropus slog getbest $base.sqlite3 --output {_output}

In [ ]:
%sosrun train-skew

In [ ]:
[predict-skew]
input: 'skewtrain.pth'
sh: expand=True
    vocropus ocroskew correct tess.tar --nsamples 10 --model skewtrain.pth

In [ ]:
%sosrun predict-skew

# Page Scale Analysis

Document layout analysis, and to a lesser degree text recognition, assumes that documents are presented at a fairly consistent image resolution (usually 300 dpi). The `ocroscale` command can be trained on a collection of documents at the desired target resolution (e.g., 300 dpi) and when applied to novel documents, will estimate the document scale relative to the training collection. No other ground truth is necessary.

The `ocroscale` command has the option for page-by-page "scale correction", though for bulk processing, it is better to estimate the scale factor for a new batch of documents once and then rescale the entire document batch with a consistent scale.

In [ ]:
[train-scale]
input: 'tess.tar'
output: 'scaletrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    base=$(basename {_output} .pth)
    rm -f $base.sqlite3
    vocropus ocroscale train {_input} --display 15 --output $base.sqlite3 --nsamples {nscaletrain} --replicate 8 --num-workers 8
    vocropus slog getbest $base.sqlite3 --output {_output}

In [ ]:
%sosrun train-scale

In [ ]:
[predict-scale]
input: 'scaletrain.pth'
sh: expand=True
    vocropus ocroscale hist tess.tar --nsamples 10 --model scaletrain.pth

In [ ]:
%sosrun predict-scale

# Document Binarization Models

For most documents, it is best to first convert them to binary format. Binarization models can be trained and applied with the `ocrobin` command.

Here, we are training the binarization model on the output of the `nlbin` command. While this may seem redundant, the `ocrobin` output is actually slightly better, and more importantly, the `ocrobin` binarizer runs on the GPU and is faster.

In [ ]:
[train-bin]
input: 'raw.tar'
output: 'bintrain.pth'
sh: expand=True
    export MPLBACKEND=TkAgg
    base=$(basename {_output} .pth)
    rm -f $base.sqlite3
    vocropus ocrobin train {_input} --display 15 --output $base.sqlite3 --nsamples {nbintrain} --extensions 'jpg nrm.jpg' --lr 1e-3 --replicate 8 --num-workers 8
    vocropus slog getbest $base.sqlite3 --output {_output}

In [ ]:
%sosrun train-bin